In [ ]:
# import pd,np,plt,sns,klib,imblearn

import pandas as pd     
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import klib as kb

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

for_later = test_df['ID']

In [ ]:
kb.missingval_plot(train_df)

In [ ]:
# import ce traget encoder

from category_encoders import TargetEncoder
#encode "Brewery" with target encoder
te = TargetEncoder()
train_df['Brewery'] = te.fit_transform(train_df['Brewery'],train_df['Style'])
test_df['Brewery'] = te.transform(test_df['Brewery'])
        

In [ ]:
kb.corr_plot(train_df,method='spearman')

In [ ]:
# droping all textual columns
train_df.drop(['Description','ID','Name','Beer Name (Full)'], axis=1, inplace=True)
test_df.drop(['W','Description','ID','Name','Beer Name (Full)'], axis=1, inplace=True)

In [ ]:
# methode 2 to fill nulls

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
train_df.iloc[:,1:] = imputer.fit_transform(train_df.iloc[:,1:])
test_df.iloc[:,1:] = imputer.fit_transform(test_df.iloc[:,1:])

In [ ]:
# filling nulls and replacing the weird 9876 walue with mean per style


for i in train_df.index :
    for j in train_df.iloc[:,1:].columns:
        if train_df.loc[i,j] == 9876 :
            #print('Before : ',train_df.loc[i,j])
            train_df.loc[i,j] = train_df.loc[(train_df['Style']==train_df.loc[i,'Style'])&(train_df[j] != 9876),:][j].mean()
            #print('After : ',train_df.loc[i,j])   
            #print('Group by was : ',train_df.loc[(train_df['Style']==train_df.loc[i,'Style'])&(train_df[j] != 9876),:][j])    
            
for i in test_df.index :
    for j in test_df.iloc[:,1:].columns:
        if test_df.loc[i,j] == 9876 :
            #print('Before : ',test_df.loc[i,j])
            test_df.loc[i,j] = test_df.loc[(test_df[j] != 9876),:][j].mean()
            #print('After : ',test_df.loc[i,j])   
            #print('Group by was : ',train_df.loc[(train_df['Style']==train_df.loc[i,'Style'])&(train_df[j] != 9876),:][j])

# Note For Fady :

i tried the modeling directly without deep feature selection, most of columns are usless, i will try smote oversampling and PCA , but the processing i made above is good, so you can use it directly to fill nulls 

In [ ]:
kb.corr_plot(train_df,method='spearman')

In [ ]:
# standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df.iloc[:,1:-1] = scaler.fit_transform(train_df.iloc[:,1:-1])
test_df.iloc[:,1:-1] = scaler.fit_transform(test_df.iloc[:,1:-1])



In [ ]:
# pca for dimensionality reduction 

from sklearn.decomposition import PCA
pca = PCA(n_components=6)
train_x_pca = pca.fit_transform(train_df.iloc[:,1:-1])
test_x_pca = pca.fit_transform(test_df.iloc[:,1:])    



In [ ]:
# xgboost model 

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV




X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]
print(X.shape,y.shape)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X,y)
print(X.shape,y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)








xgb_model = xgb.XGBClassifier(objective="multi:softmax",max_depth=3,min_child_weight=3)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))


x = xgb_model.predict(test_df.iloc[:,:])

# saving the predictions to a csv file : a dataframe with 2 columns : ID and Style
df = pd.DataFrame({'ID':for_later,'Style':x})
df.to_csv('submission.csv',index=False)
    


In [ ]:
#grid search


#param_test1 = {
# 'max_depth':range(3,10,2),
# 'min_child_weight':range(1,6,2)
#}
#gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax', num_class=104, nthread=4, scale_pos_weight=1, seed=27),param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
#gsearch1.fit(X,y)
#gsearch1.best_params_, gsearch1.best_score_
#

In [ ]:
import lightgbm as lgb
d_train=lgb.Dataset(X_train, label=y_train)
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' 
params['objective']='multiclass' 
params['metric']='multi_logloss' 
params['max_depth']=7
params['num_class']=17
params['verbose']=-1

clf=lgb.train(params,d_train,1000)  

y_pred_1=clf.predict(X_test)

from sklearn.metrics import f1_score
print(f1_score(y_test,np.argmax(y_pred_1, axis=-1), average='macro'))


# saving the predictions to a csv file 

df = pd.DataFrame({'ID':for_later,'Style':np.argmax(clf.predict(test_df.iloc[:,:]), axis=-1)})
df.to_csv('submission.csv',index=False)
    

In [ ]:
# grid search for lightgbm


param_test1 = {
    'max_depth':range(3,10,2),
    'num_leaves':range(10,100,10)
}

gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.03, n_estimators=1000, objective='multiclass',num_class=17,metric='multi_logloss',boosting_type='gbdt',verbose=-1),      param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4, cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_
    

In [ ]:
pd.read_csv('/kaggle/working/sample_submission.csv')

In [ ]:
# create the model for  the decision tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

    
    
dt_model = DecisionTreeClassifier(max_depth= 9,min_samples_split=8 )
dt_model.fit(X_train, y_train)

        
y_pred = dt_model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)


# svae to submission file 2

x = dt_model.predict(test_df.iloc[:,1:])
df = pd.DataFrame({'ID':for_later,'Style':x})

df.to_csv('submission2.csv',index=False)

accuracy

In [ ]:
print(df.shape)